In [ ]:
from component.ExtracaoAmenidadesComponent import ExtracaoAmenidadesComponent
from model.constants.ParametrosConstantes import ParametrosConstantes
from service.FeicaoOSMService import FeicaoOSMService
from service.MunicipioService import MunicipioService
from util.BancoDadosUtil import BancoDadosUtil
from util.DataFrameUtil import DataFrameUtil
from util.LoggerUtil import LoggerUtil
from util.OSUtil import OSUtil
from util.OSMNXUtil import OSMNXUtil
from util.WarningUtil import WarningUtil

from pandas import DataFrame
from time import sleep

In [ ]:
# Configurações iniciais

OSMNXUtil.configurar_osmnx()

WarningUtil.ignorar_warning(categoria=DeprecationWarning)
WarningUtil.ignorar_warning(categoria=UserWarning)

log = LoggerUtil.configurar_logger(arquivo_log=ParametrosConstantes.CAMINHO_LOG_EXTRACAO_AMENIDADES)

In [ ]:
extracao_amenidades_component = ExtracaoAmenidadesComponent()
feicao_osm_service = FeicaoOSMService()
municipio_service = MunicipioService()

In [ ]:
while True:
    log.info(msg="Iniciando o processo de extração das amenidades dos municípios.")
    conexao_bd = BancoDadosUtil.estabelecer_conexao_banco()

    if extracao_amenidades_component.verificar_nao_existencia_registros_pendentes(conexao_bd):
        log.info(msg="Encerrando a conexão com o banco de dados.")
        BancoDadosUtil.encerrar_conexao_banco(conexao_bd)
        
        break
    
    log.info(msg=f"Buscando os próximos {ParametrosConstantes.BATCH_QTDE_REGISTROS_ETAPA_EXTRACAO_AMENIDADES} municípios a serem processados.")

    gdf_municipio = municipio_service.buscar_registros_pendentes_extracao_amenidades(conexao_bd)
    df_tags_osm = feicao_osm_service.buscar_tags_osm_ativas(conexao_bd)

    df_tags_osm["tag_osm"] = extracao_amenidades_component.ajustar_coluna_tag_osm(df_tags_osm)

    log.info(msg="Extraindo as amenidades dos municípios.")

    df_resultado = DataFrameUtil.processar_dataframe_dask(
        df=gdf_municipio,
        funcao=extracao_amenidades_component.processar_particao_dask,
        meta=DataFrame(columns=["codigo_municipio", "lista_dict_amenidade_municipio", "lista_dict_historico_erro", "status"]),
        qtde_particoes=ParametrosConstantes.BATCH_QTDE_PARTICOES_DASK_ETAPA_EXTRACAO_AMENIDADES,
        df_tags_osm=df_tags_osm
    )

    extracao_amenidades_component.persistir_resultado(df_resultado, conexao_bd)

    if ParametrosConstantes.OSMXN_USAR_CACHE and ParametrosConstantes.OSMNX_LIMPAR_CACHE:
        OSUtil.deletar_diretorio(diretorio=ParametrosConstantes.DIRETORIO_CACHE_OSMNX)

    log.info(msg="Encerrando a conexão com o banco de dados.")
    BancoDadosUtil.encerrar_conexao_banco(conexao_bd)

    sleep(10)